# MoSeq modeling
Now that we have performed PCA we can finally move onto fitting a MoSeq model! The underlying algorithm is something called a autoregressive hidden markov model (AR-HMM). Pretty much, the way the model works is by taking data and clustering it into dynamical systems that evolve across short periods of time. See below for resources on what they are actually doing.

### Estimating syllable duration
In MoSeq there is something called a free parameter, which is flexible and can be controlled by the user. The name of this parameter is `kappa` and it controls the average duration of each syllable. If we inspect the timescale of rat behavior with respect to that of a mouse - we can see that rats behave on a slower timescale, the duration of their syllables are far longer than that mice. This means we need to estimate a new kappa! How might we do that? 

### Random projections and changepoint scores
When Alex W and Bob were first developing MoSeq they were trying to build a sniff detector. Something that just detected when a mouse was sampling it's environment. They decided to perform something called random projections (read more here), which is largely just an unbiased way of looking at the data. What they found is that with respect to time, there is a tremendous amount of structure in mouse behavior. We can take the first derivative of those data (a filtered version of it to be precise) to get an unbiased model of rodent behavior and measure the amount of time between each amplitude in this signal. These are referred to as changepoints, which are abrupt shifts in the similarity of an animal's pose that align with syllable transitions. Below is a graphic of this, where changepoint amplitudes align with striations in random projections, the time between each striation is a syllable. 

TODO: include rps graphic

### Determening kappa
If we measure the duration of each block using the changepoint model, we can get an estimate of the timescale of rat behavior. We can permute kappa a bunch of time to try and infer which kappa best matches the changepoint distribution. A good heuristic for rats is usually whichever model has a median syllable duration of 600 ms or a mean syllable duration of ~1 second (for mice it is 300-400 ms or 600 ms respectivley). 

In [ ]:
!moseq2-model kappa-scan /path/to/_pca/pca_scores.h5 /path/to/outdir/kscan -i /path/to/moseq2-index.yaml --min-kappa 1e6 --max-kappa 1e12 
!sh /path/to/outdir/kscan/train_out.sh

# Assesing the results
Below is a set of code that loads each model, estimates the median changepoint duration, then compares that to the desired median duration

In [17]:
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from moseq2_viz.model.util import relabel_by_usage, parse_model_results, get_best_fit, labels_to_changepoints

# define function not in moseq2
def _load_models(pth):
    mdl = parse_model_results(pth)
    mdl['changepoints'] = labels_to_changepoints(mdl['labels'], fs=30)
    return mdl

In [9]:
# get all model objects 
model_path = Path('/n/groups/datta/jlove/data/rat_seq/rat_seq_paper/data/14weeks')
models = list(model_path.glob('sn_kscan/v2/*.p'))
models = [str(m) for m in models]

In [41]:
# get the models
model_results = {name: _load_models(name) for name in models}
# get their median durations
model_times = {k: np.median(v['changepoints']) for k, v in model_results.items()}
# get how far they are from the chosen median duation (.6 seconds for rats)
diff_dict = {k: (.6-v) for k,v in model_times.items()}
# get the model with the minimum diff to .6 
best_model = min(diff_dict, key=diff_dict.get)

### Fitting the final models
Now that we have kappa we can fit the final models. The usualy heuristic is to fit 100 models for 1000 iterations. This is extremely computationally intense, I'd reccomend 10 for 500 iterations if you are not running those in parallel. 

In [ ]:
!moseq2-model kappa-scan /path/to/_pca/pca_scores.h5 /path/to/outdir/models -i /path/to/moseq2-index.yaml --min-kappa fill-in-best-kappa --max-kappa fill-in-best-kappa -n 500
!sh /path/to/outdir/models/train_out.sh